In [1]:
import os

os.chdir("../..")

In [2]:
from mars.db import collections

In [12]:
from mars.db import db

In [131]:
from tqdm import tqdm_notebook as tqdm

In [44]:
collections.sentences.ensureFulltextIndex(["sentence"])

<Index of type fulltext>

In [18]:
collections.SENTENCES

'Sentences'

In [6]:
import pandas as pd

In [154]:
df_papers = pd.read_csv("data/s2orc/ai_papers_meta.csv")
titles = df_papers.title

/home/stanislaw/repos/AI-stretegies-papers-regulations-monitoring/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [68]:
def search_for(text):
    q = db.database.AQLQuery(
    f"""FOR doc in FULLTEXT({collections.SENTENCES}, "sentence", "{text}") RETURN doc"""
    )
    return q


def get_all_citations(text):
    q = search_for(text)
    docs_citing = list(set([d.source_doc_id for d in q]))
    return docs_citing

In [60]:
len(titles)

557681

In [62]:
titles_sample = titles[:1000]

In [70]:
citations = dict()
exceptions = dict()
for title in tqdm(titles_sample):
    try:
        citations[title]=get_all_citations(title)
    except Exception as e:
        exceptions[title]=e

100%|██████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:05<00:00,  4.08it/s]


In [75]:
for c in citations:
    if len(citations[c])!=0:
        print(c)

Intelligent Systems in Education
When to Use Machine Learning


In [73]:
citations[c]

[]

In [78]:
citations["When to Use Machine Learning"]

['Documents/265537', 'Documents/259745']

In [81]:
len(exceptions)

150

In [83]:
from mars.db import new_api

In [87]:
new_api.database.create_arangosearch_view(
    "sentences_view",
    properties={
        "links": {"Sentences": {"fields": {"sentence": {"analyzers": ["text_en"]}}}}
    },
)

{'global_id': 'hF7C38D81E838/32881541',
 'id': '32881541',
 'name': 'sentences_view',
 'type': 'arangosearch',
 'cleanup_interval_step': 2,
 'commit_interval_msec': 1000,
 'consolidation_interval_msec': 1000,
 'consolidation_policy': {'type': 'tier',
  'segments_min': 1,
  'segments_max': 10,
  'segments_bytes_max': 5368709120,
  'segments_bytes_floor': 2097152,
  'min_score': 0},
 'primary_sort': [],
 'primary_sort_compression': 'lz4',
 'stored_values': [],
 'writebuffer_idle': 64,
 'writebuffer_active': 0,
 'writebuffer_max_size': 33554432,
 'links': {'Sentences': {'analyzers': ['identity'],
   'fields': {'sentence': {'analyzers': ['text_en']}},
   'include_all_fields': False,
   'track_list_positions': False,
   'store_values': 'none'}}}

In [114]:
#text= "European Union"
def search_for(text):
    query = f"""FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "{text}"), "text_en")
    RETURN doc"""
    q = db.database.AQLQuery(query, rawResults=True)
    return [{'sentence':d['sentence'],'doc_id':d['source_doc_id'], 'search_query':text} for d in q]

In [123]:
def search_for_titles(titles):
    all_results = []
    for title in tqdm(titles):
        try:
            results = search_for(title)
            all_results = all_results+results
        except Exception as e:
            print(e)
    return all_results

In [121]:
pd.DataFrame(all_results)

""


In [130]:
r = search_for_titles(titles[:10000])

 16%|█████████████▎                                                                       | 1568/10000 [00:22<02:01, 69.47it/s]

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Research of "blind" spot in LVQ network"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'blind" spot in LVQ network"), "t...' at position 1:78 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'blind" spot in LVQ network"), "t...\' at position 1:78 (while parsing)', 'errorNum': 1501}


 16%|█████████████▌                                                                       | 1600/10000 [00:22<02:00, 69.48it/s]

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "An Experiment on Game Facet Combination\"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'text_en")
    RETURN doc' at position 1:110 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'text_en")\n    RETURN doc\' at position 1:110 (while parsing)', 'errorNum': 1501}


 18%|███████████████▌                                                                     | 1829/10000 [00:26<01:57, 69.74it/s]

...

 73%|█████████████████████████████████████████████████████████████▉                       | 7282/10000 [01:46<00:38, 70.89it/s]

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, ""Unvoiced speech recognition using EMG - mime speech recognition""), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Unvoiced speech recognition usin...' at position 1:66 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': "AQL: syntax error, unexpected identifier, expecting ) near 'Unvoiced speech recognition usin...' at position 1:66 (while parsing)", 'errorNum': 1501}


 74%|██████████████████████████████████████████████████████████████▉                      | 7411/10000 [01:48<00:37, 69.10it/s]

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Identifying "best bet" web search results by mining past user behavior"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'best bet" web search results by ...' at position 1:78 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'best bet" web search results by ...\' at position 1:78 (while parsing)', 'errorNum': 1501}


100%|████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:26<00:00, 68.27it/s]


In [135]:
d = pd.DataFrame(r)

In [139]:
len(d.doc_id.unique())

45

In [140]:
r2 = search_for_titles(titles[:100000])

/home/stanislaw/repos/AI-stretegies-papers-regulations-monitoring/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/100000 [00:00<?, ?it/s]

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Research of "blind" spot in LVQ network"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'blind" spot in LVQ network"), "t...' at position 1:78 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'blind" spot in LVQ network"), "t...\' at position 1:78 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "An Experiment on Game Facet Combination\"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'text_en")
    RETURN doc' at position 1:110 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'text_en")\n    RETURN doc\' at position 1:110 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view S

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "A Contrastive Analysis of the Semantic Meaning on Synonymous Classifiers "Ceng" and "Chong""), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Ceng" and "Chong""), "text_en")
...' at position 1:139 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Ceng" and "Chong""), "text_en")\n...\' at position 1:139 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "The "feature CMAC": a neural-network-based vision system for robotic control"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'feature CMAC": a neural-network-...' at position 1:70 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'feature CMAC": a neur

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Technocultural Pluralism: A "Clash of Civilizations" in Technology?"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Clash of Civilizations" in Techn...' at position 1:94 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Clash of Civilizations" in Techn...\' at position 1:94 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, ""Those look similar!" issues in automating gesture design advice"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Those look similar!" issues in a...' at position 1:66 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Those look similar!" issues in a...\' at position 1:66 (whil

...

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Classifier Measure Word Refrain of "AXA" and Its Subjectivity in Yuexi's Gan Dialect in Anhui Province"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'AXA" and Its Subjectivity in Yue...' at position 1:101 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'AXA" and Its Subjectivity in Yue...\' at position 1:101 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Foreword to the Special Issue on "GeoVision: Computer Vision for Geospatial Applications""), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'GeoVision: Computer Vision for G...' at position 1:99 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': "AQL: syntax error, unexpected identifier, expecting ) near 

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Computational Thinking, Inferential Thinking and "Big Data""), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Big Data""), "text_en")
    RETU...' at position 1:115 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Big Data""), "text_en")\n    RETU...\' at position 1:115 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Industrial Communication " A Three Step Rocket"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'A Three Step Rocket"), "text_en"...' at position 1:92 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'A Three Step Rocket"), "text_en"...\' at position 1:92 (while parsing)', 'errorNum'

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Towards recognizing "cool": can end users help computer vision recognize subjective attributes of objects in images?"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'cool": can end users help comput...' at position 1:86 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'cool": can end users help comput...\' at position 1:86 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Supporting physicians in taking decisions in clinical guidelines: the GLARE "what if" facility."), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'what if" facility."), "text_en")...' at position 1:142 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Avoiding Implementation Pitfalls of"Matrix Capsules with EM Routing"by Hinton et al"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Matrix Capsules with EM Routing"...' at position 1:101 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Matrix Capsules with EM Routing"...\' at position 1:101 (while parsing)', 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "A Truth Maintenance System" Artificial Intelligence 12"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Artificial Intelligence 12"), "t...' at position 1:93 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Artificial Intelligence 12"), "t...\' at position 1:

Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "Learning of first-Order Formulas and Inductive Logic Programming ( "Recent Dvelopments in the Theory and Applications of Machine Learning")"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Recent Dvelopments in the Theory...' at position 1:133 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': "AQL: syntax error, unexpected identifier, expecting ) near 'Recent Dvelopments in the Theory...' at position 1:133 (while parsing)", 'errorNum': 1501}
Error in:
1: FOR doc in sentences_view SEARCH ANALYZER(PHRASE(doc.sentence, "CHASE Algorithm: "Ease of Driving" Classification"), "text_en")
2:     RETURN doc.
->AQL: syntax error, unexpected identifier, expecting ) near 'Ease of Driving" Classification"...' at position 1:83 (while parsing). Errors: {'code': 400, 'error': True, 'errorMessage': 'AQL: syntax error, unexpected identifier, expecting ) near \'Eas

In [141]:
df=pd.DataFrame(r2)

In [146]:
df.doc_id.drop_duplicates()

0        Documents/261496
1        Documents/261111
4        Documents/262690
5        Documents/274783
6        Documents/266113
               ...       
12691      Documents/1440
13369    Documents/372070
13383    Documents/260262
13402    Documents/367147
13407    Documents/346819
Name: doc_id, Length: 1066, dtype: object

In [151]:
df['search_query'].drop_duplicates()

0                                           Data Science
94                   Explainable artificial intelligence
99               Implantable Cardioverter Defibrillators
100                       Natural language understanding
103                         Convolutional Neural Network
                              ...                       
14745                 Computer Vision & Image Processing
14746                        Machine Learning Approaches
14750    Artificial Intelligence and Cognitive Sciences.
14751                                  TABLE OF CONTENTS
15612            Applications Of Artificial Intelligence
Name: search_query, Length: 128, dtype: object

In [153]:
df.loc[99]

sentence                        The Court of Justice has playe...
doc_id                                           Documents/277033
search_query              Implantable Cardioverter Defibrillators
Name: 99, dtype: object

In [157]:
df_papers[df_papers['title']=='Implantable Cardioverter Defibrillators']

,Unnamed: 0,paper_id,title,authors,abstract,year,arxiv_id,acl_id,pmc_id,pubmed_id,...,outbound_citations,inbound_citations,has_outbound_citations,has_inbound_citations,has_pdf_parse,has_pdf_parsed_abstract,has_pdf_parsed_body_text,has_pdf_parsed_bib_entries,has_pdf_parsed_ref_entries,s2_url
3798,84786,26379047,Implantable Cardioverter Defibrillators,"[{'first': 'Behzad', 'middle': [], 'last': 'Gh...","In recently years, Artificial Neural Networks ...",2009.0,NaN,NaN,NaN,19172139.0,...,"['956146', '7982079', '6751805', '44343108']",[],True,False,True,0.0,1.0,1.0,1.0,https://api.semanticscholar.org/CorpusID:26379047
